In [13]:
import numpy as np
import pandas as pd
import time
from itertools import combinations
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC,NuSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
import sys
#import tensorflow as tf
#from ELM.elm import ELM
import os
import argparse
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Check if the data is eligible
def If_Elig(data):
    return (data>=5).astype(int)

# Find the position of the data
def find_pos(data,value, order):
    max_num = np.count_nonzero(data[:,0]==value)
    if order > max_num:
        return -1
    else:
        return np.nonzero(data[:,0]==value)[0][order-1]

# Model Prediction
def model_predict(model,x):
    return model.predict(x)

# Hand Written Soft Voting
def Vote(Classifier,Prediction,Score):
    leng = len(Classifier)
    sum_score = sum(Score)
    Weighted = 0
    for i in range(leng):
        Weighted += Prediction[i]*Score[i]/sum_score
    return Weighted

In [3]:
# The following def are packages training, which can be searched on sklearn

def KNN(n,x,y):
    knn = KNeighborsClassifier(n_neighbors=n, n_jobs=-1)
    knn.fit(x,y)
    return knn


def SVM(x,y):
    svc = SVC()
    svc.fit(x,y)
    return svc


def NuSVM(x,y):
    nus = [_ / 10 for _ in range(1, 11, 1)]
    for nu in nus:
        nusvc = NuSVC(nu=nu)
        try:
            nusvc.fit(x,y)
            return nusvc
        except ValueError as e:
            print("nu {} not feasible".format(nu))

def Bayesian(x,y):
    bayesian = GaussianNB()
    bayesian.fit(x,y)
    return bayesian


def DecisionTree(x,y,criterion='gini',max_depth=None, min_samples_split=2, min_samples_leaf=1, max_features=None):
    Tree = DecisionTreeClassifier(criterion=criterion,max_depth=max_depth,min_samples_split= \
                                               min_samples_split,min_samples_leaf=min_samples_leaf,
                                               max_features=max_features)
    Tree.fit(x,y)
    return Tree


def RandomForest(x,y,n_estimator=10,criterion='gini'):
    Forest = RandomForestClassifier(n_estimators=n_estimator,criterion=criterion)
    Forest.fit(x,y)
    return Forest


def LinearC(x,y):
    linear = SGDClassifier()
    linear.fit(x,y)
    return linear


def MLP(x,y,hidden_layer_size=(100,),activation='relu',solver='adam',learning_rate_init=0.01, learning_rate='adaptive'):
    if solver=='adam':
        MLP = MLPClassifier(hidden_layer_sizes=hidden_layer_size, activation=activation, solver= \
                                               solver, learning_rate_init=learning_rate_init, learning_rate=learning_rate)
    elif solver=='sgd':
        MLP = MLPClassifier(hidden_layer_sizes=hidden_layer_size, activation=activation, solver= \
            solver, learning_rate_init=learning_rate_init)
    else:
        MLP = MLPClassifier(hidden_layer_sizes=hidden_layer_size, activation=activation, solver= \
            solver)
    MLP.fit(x,y)
    return MLP

In [8]:
# Read Data
np.set_printoptions(threshold=sys.maxsize)
cleaned_filepath = 'YSET_Dec2008_Dec2019_cleaned20190531.csv'
df = pd.read_csv(cleaned_filepath)

In [14]:
dfr = df.loc[:, 'UniqueID':'ij56_ever_combo']
d = dfr.convert_objects(convert_numeric=True)
xr = d.values
x = xr[~np.isnan(xr).any(axis=1)]
x_revise = np.empty((0,40))
y_revise = np.empty((0,17))

for i in range(x.shape[0]):
    if i == 0 and x[i,0] == x[i+1,0]:
        x_revise = np.vstack([x_revise,[x[i,0:40]]])
        y_revise = np.vstack([y_revise,[x[i,40:57]]])
    elif i == x.shape[0]-1 and x[i,0] == x[i-1,0]:
        x_revise = np.vstack([x_revise,[x[i,0:40]]])
        y_revise = np.vstack([y_revise,[x[i,40:57]]])
    elif i == x.shape[0]-1:
        continue
    elif x[i,0]==x[i+1,0] or x[i,0]==x[i-1,0]:
        x_revise = np.vstack([x_revise,[x[i,0:40]]])
        y_revise = np.vstack([y_revise,[x[i,40:57]]])
new_x = np.zeros((x_revise.shape[0],9))
for i in range(x_revise.shape[0]):
    new_x[i,0] = x_revise[i,0]
    new_x[i,1] = np.sum(x_revise[i,1:7])
    new_x[i,2] = np.sum(x_revise[i,7:10])
    new_x[i,3] = np.sum(x_revise[i,10:17])
    new_x[i,4] = np.sum(x_revise[i,17:21])
    new_x[i,5] = np.sum(x_revise[i,21:27])
    new_x[i,6] = np.sum(x_revise[i,27:32])
    new_x[i,7] = np.sum(x_revise[i,32:38])
    new_x[i,8] = np.sum(x_revise[i,38:40])

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [14]:
type(xr)

numpy.ndarray